# Audit BigQuery jobs querying data in a cross-project setup

Author: Luis Gerardo Baeza

Dec 2, 2024

Rationale: For large companies with tons of projects, they might need to audit which projects have dependencies between each other. This sample code provide an example of how to achieve this auditing.

**Use with caution.** Make sure performance, costs, and security implications are aligned with your organization policies

![audit_bq_xproj](audit_bq_xproj.png)

**Before you run:**
* Make sure you are familiar with [Audit Logs in BigQuery](https://cloud.google.com/bigquery/docs/introduction-audit-workloads)
* You must [configure a sink in Cloud Logging](https://cloud.google.com/logging/docs/export/configure_export_v2#creating_sink) to send Logs to a table in BigQuery.
* Replace **#YOUR_OWN_AUDI_TABLE#** twice with the table ID of your BigQuery table that holds the Audit data. 


In [ ]:
%%bigquery

SELECT 
    logs.timestamp
    , resource.labels.project_id source_project
  , tables.projectId target_project
  , tables.tableId
  , protopayload_auditlog.resourceName jobId
  , protopayload_auditlog.authenticationInfo.principalEmail requestor
from `#YOUR_OWN_AUDI_TABLE#` logs,
unnest(protopayload_auditlog.servicedata_v1_bigquery.jobGetQueryResultsResponse.job.jobStatistics.referencedTables) as tables
WHERE logs.timestamp >= ( CURRENT_TIMESTAMP() - INTERVAL 30 minute )
and logs.resource.type = "bigquery_resource"
and logs.protoPayload_auditlog.serviceName = "bigquery.googleapis.com"
and logs.protoPayload_auditlog.methodName = "jobservice.getqueryresults" # Only get query result method
and tables.projectId != resource.labels.project_id # Gets cross-project queries
and tables.tableId != '#YOUR_OWN_AUDI_TABLE#' # Avoids retrieving queries to the audit log table
order by timestamp desc